In [ ]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import pickle

In [ ]:
import scipy as sp
import scipy.ndimage

sigma_y = 1.5
sigma_x = 1.5
sigma = [sigma_y, sigma_x]


In [ ]:
# EVALUATION PLUME
# evaluation in water? water_dye = True
water_dye = False
# threshold for smoke *evaluation*
threshold_map = 5
# coarse is 20, fine is 10
coarse = False

# POLICY PLUME
# FSC learned in Dye in Water experimental data: WD_plume = True
WD_plume = False
# Threshold for smoke *learning*
threshold_pi = 15

smoother = True
bernoulli = True

In [ ]:
if water_dye:
    
    with open(f"new_data_bool_th50_waterdye.pkl", "rb") as f:
        frames_bool = pickle.load(f)

    with open(f"new_data_num_th50_waterdye.pkl", "rb") as f:
        frames_num = pickle.load(f)

    with open(f"new_data_av_th50_waterdye.pkl", "rb") as f:
        exp_plume = pickle.load(f)
        
    if smoother:
        exp_plume = sp.ndimage.filters.gaussian_filter(exp_plume, sigma, mode='constant')


    maxt = len(frames_num)
    exp_x0, exp_y0 = np.unravel_index(exp_plume.argmax(), exp_plume.shape)
    exp_x0 += 0 # 2
    exp_lx, exp_ly = frames_num[0].shape

        
else:

    istep = 10

    coarse = False
    if istep == 20:
        coarse = True

    with open(f"new_data_num_th{threshold_map}_istep{istep}_tstep2_minF1500_maxF5000.pkl", "rb") as f:
        frames_num = pickle.load(f)

    with open(f"new_data_bool_th{threshold_map}_istep{istep}_tstep2_minF1500_maxF5000.pkl", "rb") as f:
        frames_bool = pickle.load(f)

    with open(f"new_data_av_th{threshold_map}_istep{istep}_tstep2_minF1500_maxF5000.pkl", "rb") as f:
        exp_plume = pickle.load(f)

    if smoother:
        exp_plume = sp.ndimage.filters.gaussian_filter(exp_plume, sigma, mode='constant')
        
    maxt = len(frames_num)
    exp_x0 = np.argmax(frames_num[0][:,0])
    exp_y0 = 1
    exp_lx, exp_ly = frames_num[0].shape
    

In [ ]:
def give_obs(x, y, Lx0, Ly0, t, water_dye=False, bernoulli=False):
    true_x =     x - Lx0 + exp_x0
    if water_dye:
        true_y = (Ly - y - 1)  
    else:
        true_y = - ( y - Ly0 + 1)
    t = t%maxt
    
    #print(f"x{x} y{y} true_x{x} true_y{y}")
    #plt.imshow(frames_bool[t])
    #plt.scatter(true_y, true_x)
    #plt.show()
    
    if bernoulli:

        if (true_x >= 0) and (true_x < exp_lx):
            if (true_y >= 0) and (true_y < exp_ly):
                r = np.random.rand()
                
                return (r < exp_plume[true_x, true_y])*1, (r < exp_plume[true_x, true_y])*(th_obs+1)
            else:
                return 0, 0
        else :
            # outside of real data
            return 0, np.random.randint(3)

        
    else:

        if (true_x >= 0) and (true_x < exp_lx):
            if (true_y >= 0) and (true_y < exp_ly):
                return frames_bool[t][true_x, true_y], abs(frames_num[t][true_x, true_y])
            else:
                return 0, np.random.randint(3)
        else :
            # outside of real data
            return 0, np.random.randint(3)
    
    
    

## FSC trajectories


In [ ]:
from scipy.special import softmax as softmax

def one_step(a, x, y, Lx, Ly, A, clamp=False):
    m = a//A
    newx = x
    newy = y
    if (a%A == 0):
        newx = x-1
    if (a%A == 1):
        newx = x+1
    if (a%A == 2):
        newy = y+1
    if (a%A == 3):
        newy = y-1
    if clamp:
        newx = np.clip(newx, 0, Lx-1)
        newy = np.clip(newy, 0, Ly-1)
    return m, newx, newy


In [ ]:
#MANUAL INPUT + DEFAULT INPUTS
M=1                #memory states [Available:1,2,3,4]
A=4                #action size 4 -> [up,left,right,up]
O = 2              # distinct observations for actions [0, 1, 2, .., O-1]

if water_dye:
    # water_dye
    Lx = 92            
    Ly = 131
    Lx0 = (Lx//2)
    #Lx0 = 51
    Ly0 = 110
    find_range = 2.5
else:
    # exp_data_from_paper
    Lx = 92            
    Ly = 131
    Lx0 = (Lx//2)
    Ly0 = 91
    find_range = 2.5


#combined action space = M * A, lM0, lM1, ... rM0, rM1, ..., uM0, uM1, ..., dM0, dM1, ...
a_size = A * M
L = Lx*Ly

Neps = 1000
Tmax = 20000
fixed_time = True
boundaries = True

In [ ]:
plt.imshow(exp_plume, vmin=0, vmax=1)
    
def create_plume_from_exp(Lx, Ly, Lx0, Ly0, exp_plume_mat):

    Lx0 = int(Lx0)
    Ly0 = int(Ly0)
    center_x, center_y = np.unravel_index(np.argmax(exp_plume_mat, axis=None), exp_plume_mat.shape)
    # symmetrize exp_plume_mat
    
    min_size = min(center_x, exp_plume_mat.shape[0]-center_x)
    exp_plume_mat = exp_plume_mat[center_x - min_size:center_x + min_size,:].copy()
    
    # padding into shape
    
    new_size_x = Lx
    new_size_y = Ly

    exp_Lx, exp_Ly = exp_plume_mat.shape 
    
    assert Ly0 <= exp_Ly, 'Size of experimental plume too small' 
    
    new_plume = np.zeros((new_size_x, new_size_y))
    if Lx0 < min(exp_Lx-center_x, center_x):
        print(center_x-(Lx0), center_x+(new_size_x-Lx0),0, Ly0)
        momo = exp_plume_mat[center_x-(Lx0):center_x+(new_size_x-Lx0), 0:Ly0].copy()
        
    else:
        momo = np.zeros((new_size_x, Ly0))
        momo[Lx0-center_x:Lx0-center_x+exp_Lx,:Ly0] = exp_plume_mat[:,:Ly0].copy()
    
    new_plume[:,new_size_y-Ly0:] = momo.copy()
    
    return (new_plume.T)[::-1,:]

def create_PObs_RR(Lx, Ly, Lx0, Ly0,
                   max_obs, exp_plume_mat):

    spacex = np.arange(1,Lx+1)-(Lx+1)/2.
    spacey = np.arange(Ly)-(Ly0-1)
    cplume = create_plume_from_exp(Lx, Ly, Lx0, Ly0, exp_plume_mat)

    # -----------

    cplume = cplume[:,:].reshape(-1)  
    PObs = np.zeros((max_obs, Lx * Ly))

    fact = 1
    for i in range(max_obs-1):
        PObs[i] = cplume**i * np.exp(-cplume) / fact 
        fact = fact * (i+1)

    PObs[-1,:] = 1 - np.sum(PObs[:-1,:], axis=0)

    return np.abs(PObs) 

PlumeReal = create_PObs_RR(Lx, Ly, Lx0, Ly0, O, exp_plume)

rho0 = np.zeros(Lx*Ly)
rho0[:Lx] = PlumeReal[1,:Lx] / np.sum(PlumeReal[1,:Lx])

In [ ]:
# FSC MODEL UPLOAD
co=0
if coarse:
    co=1
if WD_plume:
    name_folder = f'./saved_policies/WDPlume_A{A}M{M}LX{Lx}LY{Ly}'
    name_FSC = name_folder + "/file_theta.out"
else:
    name_FSC = f"./saved_policies/A{A}M{M}TH{threshold_pi}sub{sub}co{co}.out"

th = np.loadtxt(name_FSC)
th = th.reshape(O, M, a_size)

pi = softmax(th, axis=2)

In [ ]:
trj_dics_FSC = []
times_FSC = np.empty(0)
for i in range(Neps):
    print(f"Trajectory {i+1} of {Neps}", end='\r')
    s0 = np.random.choice(Lx*Ly, p=rho0[:Lx*Ly])
    R = ( s0//Lx, s0%Lx)
    
    m = 0
    
    t = 0 
    y, x = R
    
    o, c = give_obs(x, y, Lx0, Ly0, t, water_dye, bernoulli=bernoulli)
    
    done = False
    found = False
    
    trjs = [R]
    trjo = [o]
    trjc = [c]
    trjm = [m]
    trja = []
    
    while (not done):

        t += 1
        
        # 
        a = np.random.choice(a_size, p=pi[o,m,:])
        m, x, y = one_step(a, x, y, Lx, Ly, A, clamp=boundaries)
        
        R = (y, x)
        
        o, c = give_obs(x, y, Lx0, Ly0, t, water_dye, bernoulli=bernoulli)
            
        if (x+1-Lx0)**2 + (y+1-Ly0)**2 < find_range**2: found = True

        trjs.append(R)
        trjo.append(o)
        trjc.append(c)
        trja.append(a)
        trjm.append(m)

            
        if found:
            done = True
            times_FSC = np.append(times_FSC, t)
        
        if fixed_time:
            if t == Tmax:
                done = True
                times_FSC = np.append(times_FSC, t)
        elif np.random.rand()<1-gamma:
            done = True
            times_FSC = np.append(times_FSC, t)
    trjdic = {'observations': np.array(trjc),
             'actions': np.array(trja),
             'timesteps': np.arange(t),
             'positions': np.array(trjs),
             'memories': np.array(trjm)}
    trj_dics_FSC.append(trjdic) 

In [ ]:
#for trj in trj_dics:
#    print(trj["rewards"][-1], trj["timesteps"][-1])
print(f"\t mean T: {times_FSC[times_FSC<Tmax-1].mean()}, \tstd: {times_FSC[times_FSC<Tmax-1].std()/np.sqrt(1000)} \tacc: {sum(times_FSC < Tmax) / Neps}")

plt.hist(times_FSC)

In [ ]:
from matplotlib import colors
                            # green                          royalblue
cmap = colors.ListedColormap(['#2E8B57', '#FFD700',  '#8B0000', 'blue'])

cdict = {1: '#D8A303', 4: '#0F984F', 2: '#A71414', 0:'blue',3:'lightgreen'}
cmap = colors.ListedColormap(['blue', '#D8A303', '#A71414', 'lightgreen'])
#cmap = colors.ListedColormap(['#0F984F','#D8A303', '#A71414'])  #M=3th15
if water_dye:
    th_obs = 50
else:
    th_obs = threshold_map
    
if water_dye:
    xlim=(0, Lx)
    ylim=(Ly, 0)
else:
    xlim=(4, Lx+4)
    ylim=(Ly0, Ly0-Ly)
    
for ep in range(10):

    dic = trj_dics_FSC[ep]
    ss = dic['positions']
    oo = (dic['observations'] >= th_obs) * 1
    mm = dic['memories']
    T = len(ss)
    
    for i in range(T-1,T):
        ss_t = ss[:i+1]
        ss_one = ss_t[oo[:i+1]>0,:]
        ss_zero = ss_t[oo[:i+1]==0,:]
        mm_t = mm[:i+1]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

        if water_dye:
            plt.setp( (ax1, ax2), xticks= np.arange(0,90,20), xticklabels=np.arange(0,90,20) )
            plt.setp( (ax1, ax2), yticks= np.arange(0,130,20), yticklabels=np.arange(0,130,20) )  
        else:
            plt.setp( (ax1, ax2), xticks= np.arange(4,94,20), xticklabels=np.arange(0,90,20) )
            plt.setp( (ax1, ax2), yticks= np.arange(-40,90,20), yticklabels=np.arange(0,130,20) )
        
        ax1.set_xlim(xlim)
        ax1.set_ylim(ylim)
        
        ax1.scatter(ss_one[:,1] - Lx0 + exp_x0, Ly0 + exp_y0 - ss_one[:,0] , c='black', s=30)
        
        ax1.scatter(ss_t[:,1] - Lx0 + exp_x0 , Ly0 + exp_y0- ss_t[:,0] , cmap=cmap, c = mm_t, s=5)
        ax1.imshow((frames_bool[i%maxt].T), origin='lower',  cmap='Greys', vmax=1.75)
        
        ax2.set_xlim(xlim)
        ax2.set_ylim(ylim)
        ax2.imshow((frames_bool[i%maxt].T), origin='upper', cmap='Greys', vmax=1.75)
        
        ax2.scatter(ss_one[:,1] - Lx0 + exp_x0, Ly0 + exp_y0 - ss_one[:,0] , c='red', s=5)
        ax2.scatter(ss_zero[:,1] - Lx0 + exp_x0 , Ly0 + exp_y0- ss_zero[:,0] , c='silver', s=5)
        ax1.scatter(48, Ly-Ly0)
        
        plt.show()

In [ ]:
- Lx0 + exp_x0

# Balkovsky algorithm

In [ ]:
class BalkosvskyShraiman:
    def __init__(self):
        
        # Initialize time
        self.time = 0
        self.count = 0
        self.state = +1 # +1 "right", 0 "left"
                    
    def action(self, obs):
        if (obs > 0):
            self.time = 1
            self.count = 0
            a = 2 # "up"
        else:
            if self.count > self.time:
                self.state = 1 - self.state
                a = 2 # "up"
                self.time += 2
                self.count = 0
            else :
                a = self.state
                self.count += 1
        return a

In [ ]:
trj_dics_FSC = []
times_FSC = np.empty(0)
for i in range(Neps):
#for i in range(20):

    print(f"Trajectory {i+1} of {Neps}", end='\r')
    s0 = np.random.choice(Lx*Ly, p=rho0[:Lx*Ly])
    R = ( s0//Lx, s0%Lx)
    
    m = 0
    
    t = 0 
    y, x = R
    
    o, c = give_obs(x, y, Lx0, Ly0, t, water_dye)
    
    fly = BalkosvskyShraiman()
    
    done = False
    found = False
    
    trjs = [R]
    trjo = [o]
    trjc = [c]
    trjm = [m]
    trja = []
    trjt = []
    
    fly_time_max = 0
    
    while (not done):

        t += 1
        
        # 
        a = fly.action(o)
        m, x, y = one_step(a, x, y, Lx, Ly, A, clamp=boundaries)
        
        R = (y, x)
        
        o, c = give_obs(x, y, Lx0, Ly0, t, water_dye)
            
        if (x+1-Lx0)**2 + (y+1-Ly0)**2 < find_range**2: found = True

        trjs.append(R)
        trjo.append(o)
        trjc.append(c)
        trja.append(a)
        trjm.append(m)
        trjt.append(fly.time)

        if fly.time > fly_time_max:
            fly_time_max = fly.time
            
        if found:
            done = True
            times_FSC = np.append(times_FSC, t)
        
        if fixed_time:
            if t == Tmax:
                done = True
                times_FSC = np.append(times_FSC, t)
        elif np.random.rand()<1-gamma:
            done = True
            times_FSC = np.append(times_FSC, t)
    trjdic = {'observations': np.array(trjc),
             'actions': np.array(trja),
             'timesteps': np.arange(t),
             'positions': np.array(trjs),
             'memories': np.array(trjm),
             'times': np.array(trjt)}
    trj_dics_FSC.append(trjdic) 

In [ ]:
#for trj in trj_dics:
#    print(trj["rewards"][-1], trj["timesteps"][-1])
print(f"\t mean T: {times_FSC[times_FSC<Tmax-1].mean()}, \tstd: {times_FSC[times_FSC<Tmax-1].std()/np.sqrt(1000)} \tacc: {sum(times_FSC < Tmax) / Neps}")

plt.hist(times_FSC)

In [ ]:
from matplotlib import colors
                            # green                          royalblue
cmap = colors.ListedColormap(['#2E8B57', '#FFD700',  '#8B0000', 'blue'])

if water_dye:
    th_obs = 50
else:
    th_obs = threshold_map

#for ep in range(1,2):
for ep in range(Neps):

    dic = trj_dics_FSC[ep]
    ss = dic['positions']
    oo = (dic['observations'] >= th_obs) * 1
    mm = dic['memories']
    T = len(ss)

    for i in range(T-1,T):
    #for i in range(500,T,5):
        ss_t = ss[:i+1]
        ss_one = ss_t[oo[:i+1]>0,:]
        ss_zero = ss_t[oo[:i+1]==0,:]
        mm_t = mm[:i+1]
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        
        ax1.imshow((frames_bool[i%maxt].T), origin='upper',  cmap='Greys')
        ax2.imshow((frames_bool[i%maxt].T), origin='upper', cmap='Greys')
        
        ax2.scatter(ss_one[:,1] - Lx0 + exp_x0, Ly0 + exp_y0 - ss_one[:,0] , c='red', s=5)
        ax2.scatter(ss_zero[:,1] - Lx0 + exp_x0 , Ly0 + exp_y0 - ss_zero[:,0] , c='silver', s=5)
        ax1.scatter(ss_t[:,1] - Lx0 + exp_x0 , Ly0 + exp_y0 - ss_t[:,0] , cmap=cmap, c = mm_t, s=5)
        #ax1.scatter(Lx0+2, Ly - Ly0, s=30)
        
        
        plt.show()

In [ ]:
ss_one

### Infotaxis

In [ ]:
from scipy.stats import entropy

class InfoTaxis:
    def __init__(self, Plume, dims, R0, stay_flag=False):
        
        # Initialize world
        self.Plume = Plume
        self.O, _, _ = Plume.shape
        self.Ly, self.Lx = dims
        
        # Initialize belief
        self.belief = np.ones( dims )
        self.belief /= np.sum(self.belief)
        self.R = R0
        
        self.belief[self.R] = 0
        
        self.stay_flag = stay_flag
        
    def information_S(self, newR):
        O, _, _ = self.Plume.shape
        y, x = newR 
        LH = self.Plume[:, y+self.Ly:y:-1, x+self.Lx:x:-1].copy()
        LH[:, y, x] = 0
        LHB = LH * self.belief[np.newaxis,:,:]
        LHO = np.sum(LHB, axis=(1,2))
        new_belief_obs = (LHB / LHO[:, np.newaxis, np.newaxis]).reshape(self.O, -1)
        entropies = entropy(new_belief_obs, axis = 1)
        newS = LHO @ entropies
        return newS
    
    def choose_action(self):
        S0 = entropy(self.belief.reshape(-1))
        # possible actions.
        dS_actions = np.empty(5)
        dS_actions[:] = +np.inf
        # actions are left right up down stay 
        # can I go left?
        y, x = self.R
        if (x > 0):
            newR = (y, x-1)
            b_newR = self.belief[newR]
            dS_actions[0] = (1 - b_newR) * self.information_S( newR ) - S0
        if (x < self.Lx-1):
            newR = (y, x+1)
            b_newR = self.belief[newR]
            dS_actions[1] = (1 - b_newR) * self.information_S( newR ) - S0
        if (y < self.Ly-1):
            newR = (y+1, x)
            b_newR = self.belief[newR]
            dS_actions[2] = (1 - b_newR) * self.information_S( newR ) - S0
        if (y > 0):
            newR = (y-1, x)
            b_newR = self.belief[newR]
            dS_actions[3] = (1 - b_newR) * self.information_S( newR ) - S0
        if (self.stay_flag):
            dS_actions[4] = self.information_S(self.R)
        a = np.argmax(- dS_actions + np.random.rand(5)*1.e-10)
        return a
        
    def new_belief(self, R, obs):
        self.R = R
        y, x = R
        LH = self.Plume[obs, y+self.Ly:y:-1, x+self.Lx:x:-1].copy()
        LH[y,x] = 0
        LHB = LH * self.belief
        LHO = np.sum(LHB)
        self.belief = (LHB / LHO)

In [ ]:
Plume = create_PObs_RR(2*Lx+1, 2*Ly+1, Lx, Ly, O, exp_plume)

In [ ]:
def one_step_simple(a, R, dims):
  y, x = R
  newx = x
  newy = y
  Ly, Lx = dims
  if (a == 0):
    newx = max(x-1,0)
  if (a == 1):
    newx = min(x+1,Lx-1)
  if (a == 2):
    newy = min(y+1,Ly-1)
  if (a == 3):
    newy = max(y-1,0)
  return (newy, newx)

In [ ]:
trj_dics_infoT = []
times_infoT = np.empty(0)

stay_flag = False
fixed_time = True

for i in range(Neps):

    print(f"Trajectory {i} of {Neps}", end='\r')
    s0 = np.random.choice(Lx*Ly, p=rho0[:Lx*Ly])
    R = ( s0//Lx, s0%Lx)
    agent = InfoTaxis(Plume.reshape(O,2*Ly+1,2*Lx+1), (Ly,Lx), R, stay_flag=stay_flag)
    
    t = 0 
    y, x = R
    
    o, c = give_obs(x, y, Lx0, Ly0, t)
    o=1
    c=5
    # Start with initial detection.
    
    agent.new_belief(R, o)

    done = False
    found = False
    
    trjs = [R]
    trjo = [o]
    trjc = [c]
    trja = []
    #trjb = []
    
    while (not done):

        t += 1
        a = agent.choose_action()

        R = one_step_simple(a, R, ( Ly, Lx))
        y, x = R
        s = x + y*Lx
        o, c = give_obs(x, y, Lx0, Ly0, t)
        agent.new_belief(R, o)
            
        if (x+1-Lx0)**2 + (y+1-Ly0)**2 < find_range**2: found = True

        trjs.append(R)
        trjo.append(o)
        trjc.append(c)
        trja.append(a)
        #trjb.append(agent.belief)

            
        if found:
            done = True
            times_infoT = np.append(times_infoT, t)
        
        if fixed_time:
            if t == Tmax:
                done = True
                times_infoT = np.append(times_infoT, t)
        elif np.random.rand()<1-gamma:
            done = True
            times_infoT = np.append(times_infoT, t)
    #{'observations': np.array, 'rewards': np.array, 'actions': np.array, 'timesteps': np.array}
    trjdic = {'observations': np.array(trjc), 
             'actions': np.array(trja),
             'timesteps': np.arange(t),
             'positions': np.array(trjs)}#,
             #'beliefs': np.array(trjb)}
    trj_dics_infoT.append(trjdic) 

In [ ]:
#for trj in trj_dics:
#    print(trj["rewards"][-1], trj["timesteps"][-1])
print(f"\t mean T: {times_infoT[times_infoT<Tmax-1].mean()}, \tstd: {times_infoT[times_infoT<Tmax-1].std()/np.sqrt(len(times_infoT))} \tacc: {sum(times_infoT < Tmax) / len(times_infoT)}")

plt.hist(times_infoT)

In [ ]:
from matplotlib import colors
                            # green                          royalblue
cmap = colors.ListedColormap(['#2E8B57', '#FFD700',  '#8B0000', 'blue'])

cdict = {1: '#D8A303', 4: '#0F984F', 2: '#A71414', 0:'blue',3:'lightgreen'}
cmap = colors.ListedColormap(['blue', '#D8A303', '#A71414', 'lightgreen'])
cmap = colors.ListedColormap(['#0F984F','#D8A303', '#A71414'])  #M=3th15

if water_dye:
    th_obs = 50
else:
    th_obs = threshold_map
    
if water_dye:
    xlim=(0, Lx)
    ylim=(Ly, 0)
else:
    xlim=(4, Lx+4)
    ylim=(Ly0, Ly0-Ly)
    
for ep in range(Neps):

    dic = trj_dics_infoT[ep]
    ss = dic['positions']
    oo = (dic['observations'] >= th_obs) * 1
    bb = dic['beliefs']
    T = len(ss)

    #for i in range(1,T,10):
    for i in range(T-1,T):
        ss_t = ss[:i+1]
        ss_one = ss_t[oo[:i+1]>0,:]
        ss_zero = ss_t[oo[:i+1]==0,:]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

        if water_dye:
            plt.setp( (ax1, ax2), xticks= np.arange(0,90,20), xticklabels=np.arange(0,90,20) )
            plt.setp( (ax1, ax2), yticks= np.arange(0,130,20), yticklabels=np.arange(0,130,20) )  
        else:
            plt.setp( (ax1, ax2), xticks= np.arange(4,94,20), xticklabels=np.arange(0,90,20) )
            plt.setp( (ax1, ax2), yticks= np.arange(-40,90,20), yticklabels=np.arange(0,130,20) )
        
        ax1.set_xlim(xlim)
        ax1.set_ylim(ylim)
        
        ax1.scatter(ss_one[:,1] - Lx0 + exp_x0, Ly0 + exp_y0 - ss_one[:,0] , c='red', s=5)
        ax1.scatter(ss_zero[:,1] - Lx0 + exp_x0 , Ly0 + exp_y0- ss_zero[:,0] , c='silver', s=5)

        ax1.imshow((frames_bool[i%maxt].T), origin='lower',  cmap='Greys', vmax=1.75)
        
        ax2.set_xlim(xlim)
        ax2.set_ylim((Ly, 0))
        #ax2.imshow((frames_bool[i%maxt].T), origin='upper', cmap='Greys', vmax=1.75)
        ax2.imshow(bb[i-1,::-1], origin='upper', cmap='Greys')
        
        ax2.scatter(ss_one[:,1] - Lx0 + exp_x0, Ly + exp_y0 - ss_one[:,0] , c='red', s=5)
        ax2.scatter(ss_zero[:,1] - Lx0 + exp_x0 , Ly + exp_y0- ss_zero[:,0] , c='silver', s=5)
        
        
        plt.show()